In [ ]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

In [ ]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler


In [ ]:
#Create SparkSession

spark = SparkSession.builder.appName("Practice Project").getOrCreate()

25/06/11 22:40:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv


--2025-06-11 22:40:31--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104, 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14354 (14K) [text/csv]
Saving to: ‘mpg-raw.csv.1’

mpg-raw.csv.1       100%[===================>]  14.02K  --.-KB/s    in 0.001s  

2025-06-11 22:40:31 (17.9 MB/s) - ‘mpg-raw.csv.1’ saved [14354/14354]



In [ ]:
# Load dataset
df = spark.read.csv("mpg-raw.csv", header=True, inferSchema=True)

### Task 4 - Print top 5 rows of the dataset


In [ ]:
df.show(5)

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|46.6|        4|       86.0|        65|  2110|      17.9|  80|Japanese|
|44.6|        4|       91.0|        67|  1850|      13.8|  80|Japanese|
|44.3|        4|       90.0|        48|  2085|      21.7|  80|European|
|44.0|        4|       97.0|        52|  2130|      24.6|  82|European|
|43.4|        4|       90.0|        48|  2335|      23.7|  80|European|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows



In [ ]:
df.groupBy('Origin').count().orderBy('count').show()

+--------+-----+
|  Origin|count|
+--------+-----+
|    null|    1|
|European|   70|
|Japanese|   88|
|American|  247|
+--------+-----+



In [ ]:
rowcount1 = df.count()
print(rowcount1)

406


In [ ]:
df = df.dropDuplicates()

In [ ]:
rowcount2 = df.count()
print(rowcount2)

[Stage 8:===================================================>   (186 + 9) / 200]

392


In [ ]:
df = df.dropna()

In [ ]:
rowcount3 = df.count()
print(rowcount3)

[Stage 11:==================================================>   (186 + 8) / 200]

385


In [ ]:
df = df.withColumnRenamed("Engine Disp","Engine_Disp")

In [ ]:
df.write.mode("overwrite").parquet("mpg-cleaned.parquet")

[Stage 14:=======>                                               (28 + 8) / 200]25/06/11 22:41:22 WARN hadoop.MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers
25/06/11 22:41:22 WARN hadoop.MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 84.47% for 8 writers
25/06/11 22:41:22 WARN hadoop.MemoryManager: Total allocation exceeds 95.00% (906,992,014 bytes) of heap memory
Scaling row group sizes to 96.54% for 7 writers


In [ ]:
print("Part 1 - Evaluation")

print("Total rows = ", rowcount1)
print("Total rows after dropping duplicate rows = ", rowcount2)
print("Total rows after dropping duplicate rows and rows with null values = ", rowcount3)
print("Renamed column name = ", df.columns[2])

import os

print("mpg-cleaned.parquet exists :", os.path.isdir("mpg-cleaned.parquet"))

Part 1 - Evaluation
Total rows =  406
Total rows after dropping duplicate rows =  392
Total rows after dropping duplicate rows and rows with null values =  385
Renamed column name =  Engine_Disp
mpg-cleaned.parquet exists : True


In [ ]:
df = spark.read.parquet("mpg-cleaned.parquet")
rowcount4 = df.count()

In [ ]:
df.show(5)
df.printSchema()

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine_Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|32.2|        4|      108.0|        75|  2265|      15.2|  80|Japanese|
|28.0|        4|      107.0|        86|  2464|      15.5|  76|European|
|26.0|        4|      156.0|        92|  2585|      14.5|  82|American|
|25.0|        4|      104.0|        95|  2375|      17.5|  70|European|
|25.0|        4|      140.0|        75|  2542|      17.0|  74|American|
+----+---------+-----------+----------+------+----------+----+--------+
only showing top 5 rows

root
 |-- MPG: double (nullable = true)
 |-- Cylinders: integer (nullable = true)
 |-- Engine_Disp: double (nullable = true)
 |-- Horsepower: integer (nullable = true)
 |-- Weight: integer (nullable = true)
 |-- Accelerate: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Origin: string (nullable = true)



In [ ]:
# Stage - 1 Using StringIndexer convert the string column "Origin" into "OriginIndex"

from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Origin", outputCol="OriginIndex")

In [ ]:
# Stage 2 - assemble the input columns 'Cylinders','Engine_Disp','Horsepower','Weight','Accelerate','Year' into a single column "features"
assembler = VectorAssembler(
    inputCols=['Cylinders', 'Engine_Disp', 'Horsepower', 'Weight', 'Accelerate', 'Year'],
    outputCol="features"
)

In [ ]:
# Stage 3 - scale the "features" using standard scaler and store in "scaledFeatures" column
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

In [ ]:
# Stage 4 - Create a LinearRegression stage to predict "MPG"

lr = LinearRegression(featuresCol="scaledFeatures", labelCol="MPG")

In [ ]:
# Build a pipeline using the above four stages

pipeline = Pipeline(stages=[indexer, assembler, scaler, lr])

In [ ]:
# Split the data into training and testing sets with 70:30 split. Use 42 as seed
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)

In [ ]:
# Fit the pipeline using the training data

pipelineModel = pipeline.fit(trainingData)

25/06/11 22:47:00 WARN util.Instrumentation: [5e8d2f50] regParam is zero, which might cause numerical instability and overfitting.
[Stage 24:>                                                         (0 + 8) / 8]25/06/11 22:47:02 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
25/06/11 22:47:02 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
25/06/11 22:47:02 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
25/06/11 22:47:02 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [ ]:
print("Part 2 - Evaluation")
print("Total rows = ", rowcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  385
Pipeline Stage 1 =  StringIndexer
Pipeline Stage 2 =  VectorAssembler
Pipeline Stage 3 =  StandardScaler
Label column =  MPG


In [ ]:
# Make predictions on testing data
predictions = pipelineModel.transform(testingData)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="mse")
mse = evaluator.evaluate(predictions)
print(mse)


[Stage 31:>                                                         (0 + 8) / 8]

10.899168410636614


In [ ]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)



[Stage 33:>                                                         (0 + 8) / 8]

2.625684006095087


### Task 4 - Print the R-Squared(R2)


In [ ]:
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)


[Stage 35:>                                                         (0 + 8) / 8]

0.8202762026606879


In [ ]:
print("Part 3 - Evaluation")

print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = pipelineModel.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Part 3 - Evaluation
Mean Squared Error =  10.9
Mean Absolute Error =  2.63
R Squared =  0.82
Intercept =  -13.9


In [ ]:
# Save the pipeline model
# your code goes here
pipelineModel.write().save("Practice_Project")

In [ ]:
# Load the pipeline model
# your code goes here
loadedPipelineModel = PipelineModel.load("Practice_Project")

In [ ]:
# Use the loaded pipeline model for predictions
# your code goes here
predictions = loadedPipelineModel.transform(testingData)

In [ ]:
# your code goes here
predictions.select("MPG","prediction").show()

[Stage 80:>                                                         (0 + 1) / 1]

+----+------------------+
| MPG|        prediction|
+----+------------------+
|13.0| 11.31880991495555|
|13.0|14.368910120039454|
|13.0|10.684980370654229|
|15.0| 13.06659233990255|
|15.5|15.669787158219199|
|18.0| 19.81597780531922|
|18.0|22.299804385989376|
|18.0|20.053788782604926|
|18.6|20.890856761279736|
|19.0|24.862180440385856|
|21.5|26.265038242693542|
|22.0| 23.09860601340658|
|23.0| 21.27831362520896|
|24.0|22.969583765345746|
|25.1|27.030283144018128|
|26.0|27.950969788034072|
|26.0|25.754412517142313|
|27.0| 28.40010908515616|
|29.0|28.123745554548947|
|30.0| 30.24946672093104|
+----+------------------+
only showing top 20 rows



In [ ]:
print("Part 4 - Evaluation")

loadedmodel = loadedPipelineModel.stages[-1]
totalstages = len(loadedPipelineModel.stages)
inputcolumns = loadedPipelineModel.stages[1].getInputCols()

print("Number of stages in the pipeline = ", totalstages)
for i,j in zip(inputcolumns, loadedmodel.coefficients):
    print(f"Coefficient for {i} is {round(j,4)}")

In [ ]:
spark.stop()

<!--
|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-26|0.1|Ramesh Sannareddy|Initial Version Created|
-->
